In [1]:
from pathlib import Path
from collections import deque
from itertools import repeat, permutations, chain, repeat
from io import StringIO
import numpy as np
import re
from functools import cache

In [2]:
codes = Path("data/21.txt").read_text().strip().split("\n")

def ascii_to_grid(ascii_data: str):
    grid = []
    for line in ascii_data.split("\n"):
        if "|" in line:
            values = [val.strip() for val in line.split("|")[1:-1]]
            if len(values) == 2:
                values = ["#"] + values
            grid.append(values)
    return np.array(grid, dtype=object)

In [3]:
numeric_keypad = ascii_to_grid(
    """
+---+---+---+
| 7 | 8 | 9 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
    | 0 | A |
    +---+---+
""".strip()
)

numeric_lookup = {v: (i, j) for (i, j), v in np.ndenumerate(numeric_keypad) if v != "#"}

directional_keypad = ascii_to_grid(
    """
    +---+---+
    | ^ | A |
+---+---+---+
| < | v | > |
+---+---+---+
""".strip()
)

directional_lookup = {
    v: (i, j) for (i, j), v in np.ndenumerate(directional_keypad) if v != "#"
}

In [4]:

#[(-1, 0), (1, 0), (0, -1), (0, 1)]
# ^ v < >
def shortest_path(a, b, grid):
    if a == b:
        return ""
        
    a_x, a_y = a
    b_x, b_y = b
    if a_y <= b_y and a_x <= b_x:
        return '>' * (b_y - a_y) + 'v' * (b_x - a_x)
    elif a_y <= b_y and a_x > b_x:
        return '^' * (a_x - b_x) + '>' * (b_y - a_y)
    elif a_y > b_y and a_x <= b_x:
        return '<' * (a_y - b_y) + 'v' * (b_x - a_x)
    elif a_y > b_y and a_x > b_x:
        return '^' * (a_x - b_x) + '<' * (a_y - b_y)

In [5]:
def keys(code, grid, lookup, acc=""):
    if len(code) <= 1:
        yield acc
        return

    start, goal = code[0], code[1]
    if start != goal:
        yield from keys(code[1:], grid, lookup, f"{acc}{shortest_path(lookup[start], lookup[goal], grid)}A")
    else:
        yield from keys(code[1:], grid, lookup, f"{acc}A")

In [6]:
@cache
def pair_keys(p, depth):
    if depth == 0:
        return p

    return pair_keys(
        "".join(
            [
                part
                for a, b in zip(f"A{p}", p)
                for part in shortest_path(
                        directional_lookup[a], directional_lookup[b], directional_keypad
                    ) + "A"
            ]
        ),
        depth - 1,
    )

In [7]:
def extract_int_from_string(text):
    match = re.search(r"\d+", text)
    return int(match.group()) if match else None


@cache
def min_len(code, d=3):
    m = None
    for p0 in keys(code, numeric_keypad, numeric_lookup):
        p1 = pair_keys(p0, d - 1)
        m = min(m or len(p1), len(p1))
    return m


sum(
    min_len(f"{a}{b}") * extract_int_from_string(code)
    for code in codes
    for a, b in zip(f"A{code}", code)
)

211546

In [8]:
sum(
    min_len(f"{a}{b}", d=25) * extract_int_from_string(code)
    for code in codes
    for a, b in zip(f"A{code}", code)
)

KeyboardInterrupt: 